In [23]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

In [24]:
# create a MongoDB client instance
client = MongoClient('mongodb://localhost:27017')

# get a reference to the database
db = client.mydatabase

# get a reference to the collection
collection = db.closing_prices_latest

# define your Alpha Vantage API key and endpoint URL
api_key = "HMU3C6GSU9RRXY5Z"
url = "https://www.alphavantage.co/query"

# define the symbol and data type to retrieve
symbol = "AAPL"
data_type = "TIME_SERIES_DAILY_ADJUSTED"

# define the API parameters
params = {
    "function": data_type,
    "symbol": symbol,
    "apikey": api_key
}

In [25]:
# make a request to the Alpha Vantage API
response = requests.get(url, params=params)

# convert the response data to a Python dictionary
data = json.loads(response.text)

In [26]:
# insert each day's closing price as a separate document
for date, values in data["Time Series (Daily)"].items():
    document = {
        "symbol": symbol,
        "timestamp": pd.to_datetime(date),
        "close": float(values["4. close"])
    }
    collection.insert_one(document)

In [27]:
# make a request to the Alpha Vantage API
response = requests.get(url, params=params)

# convert the response data to a Python dictionary
data = json.loads(response.text)

In [28]:
# get the latest timestamp in the collection, if any
latest_timestamp = collection.find_one(sort=[("timestamp", -1)])["timestamp"] if collection.count_documents({}) > 0 else None

In [29]:
# append new data to the collection
for date, values in data["Time Series (Daily)"].items():
    timestamp = pd.to_datetime(date)
    if latest_timestamp is None or timestamp > latest_timestamp:
        document = {
            "symbol": symbol,
            "timestamp": timestamp,
            "close": float(values["4. close"])
        }
        collection.update_one({"timestamp": timestamp}, {"$set": document}, upsert=True)